<a href="https://colab.research.google.com/github/zahra-zarrabi/Pytorch_Fashion_Mnist/blob/master/fashion_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install wandb

In [ ]:
import torch
import torchvision
from tqdm import tqdm
import wandb

In [ ]:
cd 'drive/MyDrive/fashion_mnist'

/content/drive/MyDrive/fashion_mnist


In [ ]:
wandb.init(project='FashionMNIST')

In [ ]:
def calc_acc(preds,labels):
  _,pred_max=torch.max(preds,1)
  acc=torch.sum(pred_max==labels.data,dtype=torch.float64)/len(preds)
  return acc

In [ ]:
class cnn_model(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.cnn1 = torch.nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5,stride=1,padding=2)
    self.cnn2=torch.nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=2)   
    self.fc1=torch.nn.Linear(1568,10)

  def forward(self,x):
    x=self.cnn1(x)
    x=torch.relu(x)
    x=torch.nn.MaxPool2d(2, stride=2)(x)
    x=self.cnn2(x)
    x=torch.relu(x)
    x=torch.nn.MaxPool2d(2, stride=2)(x)
    x=x.reshape((x.shape[0],1568))
    x=self.fc1(x)
    return x

In [ ]:
device=torch.device('cuda')
model=cnn_model()
model = model.to(device)

In [ ]:
batch_size=64
epochs=10
lr=0.01

In [ ]:
rm -rf *

In [ ]:
#data preparing
transform=torchvision.transforms.Compose([
                                          torchvision.transforms.ToTensor(),
                                          torchvision.transforms.ConvertImageDtype(torch.float32),
                                          # torchvision.transforms.Normalize((0),(255)),
                                          torchvision.transforms.RandomHorizontalFlip()
])
train_data=torchvision.datasets.FashionMNIST('./dataset',train=True,download=True,transform=transform)
train_data_loader=torch.utils.data.DataLoader(train_data,batch_size=batch_size,shuffle=True)

test_data=torchvision.datasets.FashionMNIST('./dataset',train=False,download=True,transform=transform)
test_data_loader=torch.utils.data.DataLoader(test_data,batch_size=batch_size,shuffle=False)


In [ ]:
# compile
optimizer = torch.optim.SGD(model.parameters(),lr=lr)
loss_function=torch.nn.CrossEntropyLoss()

In [ ]:
def train(model, train_data_loader,epoch):
  model.train(True)
  train_loss=0.0
  train_acc=0.0
  for images,labels in tqdm(train_data_loader):
    images=images.to(device)
    labels=labels.to(device)
    optimizer.zero_grad()
    
    preds_train = model(images)

    loss_train=loss_function(preds_train,labels) # loss_train
    loss_train.backward()

    optimizer.step()

    train_loss += loss_train
    train_acc += calc_acc(preds_train,labels)
    
  total_loss = train_loss/len(train_data_loader)
  total_acc = train_acc/len(train_data_loader)
  print(f"loss_train:{total_loss},accuracy_train:{total_acc}")

  wandb.log({'epochs':  epoch,
              'loss_train': total_loss,
              'accuracy_train': total_acc
                              })

In [ ]:
def test(model, test_data_loader,epoch):
  model.eval()
  test_loss=0.0
  test_acc=0.0
  for images,labels in tqdm(test_data_loader):    
    images=images.to(device)
    labels=labels.to(device)
    
    preds_test = model(images)

    loss_test=loss_function(preds_test,labels) 

    test_loss += loss_test
    test_acc += calc_acc(preds_test,labels)

  total_loss = test_loss/len(test_data_loader)
  total_acc = test_acc/len(test_data_loader)
  print(f"loss_eval:{total_loss},accuracy_eval:{total_acc}")

  wandb.log({'epochs':  epoch,
              'loss_eval': total_loss,
              'accuracy_eval': total_acc
                              })

In [ ]:
for epoch in range(epochs):
  print(f'Epoch:{epoch}')
  train(model, train_data_loader,epoch)
  test(model, test_data_loader,epoch)  


Epoch:0


100%|██████████| 938/938 [00:16<00:00, 58.53it/s]


loss_train:1.0472759008407593,accuracy_train:0.6350946162046908


100%|██████████| 157/157 [00:02<00:00, 63.57it/s]


loss_eval:0.7028485536575317,accuracy_eval:0.7593550955414013
Epoch:1


...

loss_eval:0.40940016508102417,accuracy_eval:0.8556926751592357
Epoch:9


100%|██████████| 938/938 [00:15<00:00, 59.96it/s]


loss_train:0.37518876791000366,accuracy_train:0.8700692963752665


100%|██████████| 157/157 [00:02<00:00, 66.41it/s]

loss_eval:0.3972972333431244,accuracy_eval:0.8607683121019108


In [ ]:
torch.save(model.state_dict(), "fashion_mnist.pth")

In [ ]:
# test_data=torchvision.datasets.FashionMNIST('./dataset',train=False,download=True)

In [ ]:
# test_data.data.numpy().shape

In [ ]:

# cnt = 0
# for images,labels in tqdm(test_data):
#   images.save("{}.jpeg".format(labels))
#   print(labels)
#   cnt += 1
#   if cnt > 5:
#     break